In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt


import numpy as np
import sys
sys.path.append('/Users/diegofiori/Desktop/epfl/master_thesis/master_thesis/')
from input_reader import read_simulation_file, get_all_time_ids
from filter import FilterBigComponents
from matplotlib.animation import ArtistAnimation
from giotto.pipeline import Pipeline
from giotto.homology import CubicalPersistence
from giotto.diagrams import Scaler, BettiCurve, PersistenceLandscape
import matplotlib.cm as cm
from tqdm.notebook import tqdm

In [2]:
"""path = '/Users/diegofiori/Desktop/epfl/master_thesis/Reverse/'
space_index, field = 0, 'temperature'
img = [] # some array of images
time_ids = get_all_time_ids(path)
for i in tqdm(range(0, len(time_ids), 6)):
    simulation_slices = read_simulation_file(path, field, time_ids[i:i+6])
    simulation_slices = [simulation_slices[j] for j in range(0, len(simulation_slices), 80)]
    img += simulation_slices"""

"path = '/Users/diegofiori/Desktop/epfl/master_thesis/Reverse/'\nspace_index, field = 0, 'temperature'\nimg = [] # some array of images\ntime_ids = get_all_time_ids(path)\nfor i in tqdm(range(0, len(time_ids), 6)):\n    simulation_slices = read_simulation_file(path, field, time_ids[i:i+6])\n    simulation_slices = [simulation_slices[j] for j in range(0, len(simulation_slices), 80)]\n    img += simulation_slices"

In [3]:
from utils import read_pickle
img = read_pickle('/Users/diegofiori/Desktop/epfl/master_thesis/results/density2dfield.pickle')

In [4]:
def plot_slice_diagram(image, homology_dimensions=(0, 1), figure_plot=None):
    cub = CubicalPersistence(homology_dimensions=homology_dimensions)
    scaler = Scaler(metric='bottleneck')
    pipeline = Pipeline([('diagram', cub),
                         ('rescale', scaler)])
    diagram = pipeline.fit_transform(np.expand_dims(image, axis=0))

    color_dict = {0: '.r', 1: '.b', 2: '.g'}
    points = diagram[0, :, :-1]
    dims = diagram[0, :, -1]

    for hom_dim in homology_dimensions:
        hom_points = points[dims == hom_dim]
        plt.plot(hom_points[:, 0], hom_points[:, 1], color_dict[hom_dim])
    min_b, max_b = np.min(points[:, 0]), np.max(points[:, 0])
    figure_plot.plot([min_b, max_b], [min_b, max_b], 'k')

In [5]:
def create_video(list_of_images, time_series, avg_field, prec_diags=None, 
                 interval=50, repeat=True, repeat_delay=100, **fig_params):
    flag = True
    if prec_diags is None:
        pipeline = Pipeline([('cub', CubicalPersistence()), ('rescale', Scaler('bottleneck'))])
        diagrams = pipeline.fit_transform(list_of_images)
    else:
        diagrams = prec_diags
    betti_curves = BettiCurve(n_values=50).fit_transform(diagrams)
    landscapes = PersistenceLandscape(n_layers=10, n_values=50).fit_transform(diagrams)
    homology_dimensions = np.unique(diagrams[0, :, -1])
    color_dict = {0: '.r', 1: '.b', 2: '.g'}
    color_dict_land = {0: 'r', 1: 'b', 2: 'g'}
    fig = plt.figure(**fig_params)
    viewer_1 = fig.add_subplot(231)
    viewer_2 = fig.add_subplot(232)
    viewer_3 = fig.add_subplot(233)
    viewer_4 = fig.add_subplot(234)
    viewer_5 = fig.add_subplot(235)
    viewer_6 = fig.add_subplot(236)
    fig.show()
    plt.ion()
    while flag:
        try:
            for i, image in enumerate(list_of_images):
                viewer_1.clear()
                viewer_1.imshow(image)
                viewer_2.clear()
                viewer_2.plot(time_series)
                viewer_2.plot([i, i], [np.min(time_series), np.max(time_series)])
                viewer_3.clear()
                viewer_3.plot(avg_field)
                viewer_3.plot([i, i], [np.min(avg_field), np.max(avg_field)])
                viewer_4.clear()
                diagram = diagrams[i]
                points = diagram[:, :-1]
                dims = diagram[:, -1]
                for hom_dim in homology_dimensions:
                    hom_points = points[dims == hom_dim]
                    viewer_4.plot(hom_points[:, 0], hom_points[:, 1], color_dict[hom_dim])
                    min_b, max_b = np.min(points[:, 0]), np.max(points[:, 0])
                    viewer_4.plot([min_b, max_b], [min_b, max_b], 'k')
                viewer_5.clear()
                for hom_dim in homology_dimensions:
                    viewer_5.plot(np.arange(50), betti_curves[i, int(hom_dim), :], 
                                  color_dict_land[hom_dim])
                viewer_6.clear()
                for hom_dim in homology_dimensions:
                    for layer in range(10):
                        viewer_6.plot(np.arange(50), landscapes[i, int(hom_dim), layer, :], 
                                      color_dict_land[hom_dim])
                plt.pause(interval/1000)
            if repeat:
                plt.pause(repeat_delay/1000)
            else:
                flag = False
        except KeyboardInterrupt:
            flag = False

## CompSnow

In [6]:
top_features = read_pickle('/Users/diegofiori/Desktop/epfl/master_thesis/results/slices_top_features_end_cs.pickle')

In [7]:
top_features.shape

(4, 7680, 11134)

In [9]:
top_features = np.concatenate([np.expand_dims(top_features[:, i:i+80, :], axis=1) 
                               for i in range(0, top_features.shape[1], 80)], axis=1)

In [10]:
top_features.shape

(4, 96, 80, 11134)

In [11]:
top_feat_index = read_pickle('/Users/diegofiori/Desktop/epfl/master_thesis/results/selected_index_cs.pickle')

In [12]:
len(top_feat_index)

88

In [13]:
for index in top_feat_index[:3]:
    plt.figure()
    plt.plot(top_features[index[0], :, 0, index[1]])

In [14]:
index = top_feat_index[0]
time_series = top_features[index[0], :, 0, index[1]]

In [15]:
density = read_pickle('/Users/diegofiori/Desktop/epfl/master_thesis/results/physical_features_cs.pickle')

In [16]:
density = np.concatenate(density)

In [17]:
top_feat_index[0]

(2, 0)

In [18]:
density.shape

(1888,)

In [19]:
density = density[:len(img)]

In [20]:
from scipy.io import loadmat
temp = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_end_cs.mat')
time_series = temp['k_quantities']

In [21]:
time_series = time_series[:, 0, 0]

In [22]:
pipeline = Pipeline([('diag', CubicalPersistence()), 
                     ('scaler', Scaler()), 
                     ('filter', FilterBigComponents())
                    ])
precomp_diagrams = pipeline.fit_transform(img[-100:])

In [23]:
precomp_diagrams.shape

(100, 1610, 3)

In [24]:
create_video(img[-100:], time_series[-100:], density[-100:], prec_diags=precomp_diagrams, 
             interval=.1, repeat_delay=1000)

# EM_Large

In [ ]:
img_eml = read_pickle('/Users/diegofiori/Desktop/epfl/master_thesis/results/density2dfield_eml.pickle')

In [ ]:
top_features_large = read_pickle('/Users/diegofiori/Desktop/epfl/master_thesis/results/slices_top_features_em.pickle')

In [ ]:
top_features_large = np.concatenate([np.expand_dims(top_features_large[:, i:i+80, :], axis=1) 
                                     for i in range(0, top_features_large.shape[1], 80)], axis=1)

In [ ]:
for index in top_feat_index[:3]:
    plt.figure()
    plt.plot(top_features_large[index[0], :, index[1], index[2]])

In [22]:
index = top_feat_index[0]
time_series_eml = top_features_large[index[0], :, index[1], index[2]]

NameError: name 'top_features_large' is not defined

In [28]:
density_eml = read_pickle('/Users/diegofiori/Desktop/epfl/master_thesis/results/physical_features_em.pickle')
density_eml = np.concatenate(density_eml)

In [29]:
plt.plot(density_eml)
plt.show()

In [30]:
density_eml = density_eml[:len(img_eml)]

In [31]:
# create_video(img_eml[::4], time_series_eml[::4], interval=.1, repeat_delay=1000)
create_video(img_eml, time_series_eml, interval=.1, repeat_delay=1000)